# Assistente Avaliador de Notícias

## Descrição do Projeto
Este agente de IA é capaz de:
1. **Buscar notícias** na web usando DuckDuckGo
2. **Avaliar o sentimento** das notícias (positivo ou negativo)
3. **Monitorar tokens** utilizados em cada operação

## Dependências
As dependências necessárias podem ser instaladas pelo `requirements.txt` ou pelo bloco abaixo

In [1]:
%pip install --upgrade \
    python-dotenv \
    langchain \
    langchain-core \
    langchain-mistralai \
    langchain-community \
    ddgs \
    langgraph \
    transformers \
    ipywidgets \
    torch \
    torchvision \
    torchaudio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


##### Imports

In [2]:
import os
from typing import Dict, Any
from langchain_mistralai import ChatMistralAI
from dotenv import load_dotenv
from ddgs import DDGS
from langchain_core.prompts import PromptTemplate
from langchain.tools import tool

# Desabilitar paralelismo do tokenizers para evitar warning de fork
os.environ["TOKENIZERS_PARALLELISM"] = "false"

load_dotenv()

True

##### API_KEY
Adquira a chave no site https://admin.mistral.ai/organization/api-keys se necessário

In [3]:
api_key = os.getenv("MISTRAL_API_KEY")
if not api_key:
    raise ValueError("MISTRAL_API_KEY não encontrada. Configure no arquivo .env")

Definindo modelo LLM

In [4]:
llm = ChatMistralAI(
    model="open-mistral-7b",
    temperature=0,
    max_retries=1,
    verbose=True # Ajuda a ver tokens e modelo
)

## 1. Tool de Busca de Notícias

Implementação da tool que busca notícias usando DuckDuckGo

In [5]:
@tool
def buscar_noticias(query: str) -> str:
    """Busca notícias recentes na web sobre um tópico específico usando DuckDuckGo."""
    try:
        # Adiciona palavras-chave para garantir que são notícias
        query_noticias = f"{query} notícias"
        
        # Usa a API direta do ddgs para buscar notícias
        ddgs = DDGS()
        
        # Busca especificamente por notícias com região Brasil
        resultados = list(ddgs.news(
            query_noticias, 
            region='br-pt',  # Região Brasil
            safesearch='moderate',
            max_results=2
        ))
        
        if not resultados:
            return f"Nenhuma notícia encontrada sobre: {query}"
        
        # Formata os resultados
        texto_formatado = f"Encontradas {len(resultados)} notícias sobre '{query}':\n\n"
        
        for i, resultado in enumerate(resultados, 1):
            titulo = resultado.get('title', 'Sem título')
            body = resultado.get('body', 'Sem descrição')
            url = resultado.get('url', '')
            data = resultado.get('date', 'Data não disponível')
            
            texto_formatado += f"NOTÍCIA {i}:\n"
            texto_formatado += f"Título: {titulo}\n"
            texto_formatado += f"Resumo: {body}\n"
            texto_formatado += f"Data: {data}\n"
            texto_formatado += f"Fonte: {url}\n"
            texto_formatado += "-" * 80 + "\n\n"
        
        return texto_formatado
        
    except Exception as e:
        return f"Erro ao buscar notícias: {str(e)}"

print(f"Tool '{buscar_noticias.name}' criada com sucesso!")

Tool 'buscar_noticias' criada com sucesso!


## 2. Tool de Análise de Sentimento

Implementação da tool que avalia se o conteúdo é positivo ou negativo

In [6]:
@tool
def analisar_sentimento(texto: str) -> str:
    """Analisa o sentimento de um texto e determina se é positivo ou negativo. Use após obter o conteúdo de notícias."""
    try:
        prompt = PromptTemplate(
            input_variables=["text"],
            template="""Analise o sentimento do seguinte texto de uma notícia e classifique como POSITIVO ou NEGATIVO. Use Português Brasileiro e traduza o titulo da noticia para português caso não esteja.

Texto: {text}

Responda no formato:
NOTICIAS: [Coloque o titulo das noticias e nome do jornal que as publicou]
SENTIMENTO: [POSITIVO/NEGATIVO]
JUSTIFICATIVA: [explicação breve do porquê]
"""
        )
        
        chain = prompt | llm
        resultado = chain.invoke({"text": texto})
        
        # Extrair o conteúdo da resposta
        if hasattr(resultado, 'content'):
            return resultado.content
        return str(resultado)
    except Exception as e:
        return f"Erro ao analisar sentimento: {str(e)}"

print(f"Tool '{analisar_sentimento.name}' criada com sucesso!")

Tool 'analisar_sentimento' criada com sucesso!


## 3. Configuração do Agente com Monitoramento de Tokens

Criação do agente ReAct com as tools e sistema de contagem de tokens

In [7]:
# Classe para monitorar tokens
class TokenMonitor:
    def __init__(self):
        self.total_tokens = 0
        self.operacoes = []
    
    def adicionar_operacao(self, operacao: str, tokens: int):
        self.total_tokens += tokens
        self.operacoes.append({"operacao": operacao, "tokens": tokens})
    
    def obter_relatorio(self):
        relatorio = f"\n{'='*50}\n"
        relatorio += "RELATÓRIO DE USO DE TOKENS\n"
        relatorio += f"{'='*50}\n"
        for op in self.operacoes:
            relatorio += f"{op['operacao']}: {op['tokens']} tokens\n"
        relatorio += f"{'='*50}\n"
        relatorio += f"TOTAL: {self.total_tokens} tokens\n"
        relatorio += f"{'='*50}\n"
        return relatorio
    
    def resetar(self):
        """Reseta o contador de tokens"""
        self.total_tokens = 0
        self.operacoes = []

# Inicializar monitor
token_monitor = TokenMonitor()

# Criar lista de tools
tools = [buscar_noticias, analisar_sentimento]

# Criar um agente simples que chama as tools conforme necessário
class SimpleNewsAgent:
    def __init__(self, llm, tools):
        self.llm = llm
        self.tools = {tool.name: tool for tool in tools}
    
    def run(self, query: str) -> str:
        """Executa o agente para processar a query"""
        # Primeiro, busca notícias
        news_result = self.tools['buscar_noticias'].invoke(query)
        
        # Depois, analisa o sentimento
        sentiment_result = self.tools['analisar_sentimento'].invoke(news_result)
        
        # Monta resposta final
        final_response = f"""
NOTÍCIAS ENCONTRADAS:
{news_result}

ANÁLISE DE SENTIMENTO:
{sentiment_result}
"""
        return final_response

# Criar o agente
agent_executor = SimpleNewsAgent(llm, tools)

print("Agente configurado com sucesso!")
print(f"Tools disponíveis: {[tool.name for tool in tools]}")

Agente configurado com sucesso!
Tools disponíveis: ['buscar_noticias', 'analisar_sentimento']


## 4. Implementação

In [8]:
def truncar_texto(texto: str, max_chars: int = 3000) -> str:
    """
    Trunca o texto para evitar erro de sequence length do modelo.
    O modelo Mistral tem limite de ~1024 tokens (≈4000 caracteres).
    """
    if len(texto) > max_chars:
        return texto[:max_chars] + "\n\n[Texto truncado para respeitar limite do modelo]"
    return texto

def contar_tokens_aproximado(texto: str) -> int:
    """
    Conta tokens de forma aproximada (1 token ≈ 4 caracteres em português).
    Método alternativo quando transformers não está disponível.
    """
    return len(texto) // 4

def executar_consulta_com_monitoramento(query: str):
    """
    Executa uma consulta no agente e monitora o uso de tokens.
    """
    # Resetar o monitor antes de cada consulta
    token_monitor.resetar()
    
    print(f"\nConsultando: {query}\n")
    print("="*60)
    
    # Executar a consulta
    resultado = agent_executor.run(query)
    
    # Mostrar o resultado
    print(resultado)
    
    # Contar tokens da entrada
    try:
        tokens_entrada = llm.get_num_tokens(truncar_texto(query))
    except Exception:
        tokens_entrada = contar_tokens_aproximado(query)
    token_monitor.adicionar_operacao("Entrada do usuário", tokens_entrada)
    
    # Contar tokens da saída (truncar para evitar erro de sequence length)
    try:
        tokens_saida = llm.get_num_tokens(truncar_texto(resultado))
    except Exception:
        tokens_saida = contar_tokens_aproximado(resultado)
    token_monitor.adicionar_operacao("Resposta do agente", tokens_saida)
    
    # Mostrar relatório de tokens
    print(token_monitor.obter_relatorio())

#### Exemplo de busca 

In [12]:
# Exemplo 1: Buscar noticias sobre um politico
query1 = "Lula"
resultado1 = executar_consulta_com_monitoramento(query1)


Consultando: Lula




NOTÍCIAS ENCONTRADAS:
Encontradas 2 notícias sobre 'Lula':

NOTÍCIA 1:
Título: 'Todo mundo sabe o que ele fez', diz Lula sobre prisão de Bolsonaro
Resumo: O presidente Lula (PT) disse hoje na reunião com a cúpula de líderes do G20 que não comentaria decisões do STF e afirmou que ...
Data: 2025-11-24T11:17:35+00:00
Fonte: https://noticias.uol.com.br/politica/ultimas-noticias/2025/11/23/todo-mundo-sabe-o-que-ele-fez-diz-lula-sobre-prisao-de-bolsonaro.htm
--------------------------------------------------------------------------------

NOTÍCIA 2:
Título: Lula fala pela primeira vez sobre prisão de Bolsonaro: "Todo mundo sabe o que ele fez"
Resumo: O presidente Luiz Inácio Lula da Silva (PT) comentou pela primeira vez, neste domingo (23/11), a prisão preventiva do ...
Data: 2025-11-24T11:17:35+00:00
Fonte: https://www.correiobraziliense.com.br/politica/2025/11/7298214-lula-fala-pela-primeira-sobre-prisao-de-bolsonaro-todo-mundo-sabe-o-que-ele-fez.html
-------------------------------------

## 5. Mais Exemplos de Uso

Você pode testar diferentes consultas abaixo:

In [10]:
# Notícias sobre economia
query2 = "Busque notícias sobre a economia brasileira"
resultado2 = executar_consulta_com_monitoramento(query2)


Consultando: Busque notícias sobre a economia brasileira




NOTÍCIAS ENCONTRADAS:
Encontradas 2 notícias sobre 'Busque notícias sobre a economia brasileira':

NOTÍCIA 1:
Título: Mercado dá indícios de incerteza para a economia brasileira após carta de Trump
Resumo: A percepção de risco sobre a economia brasileira aumentou nesta quarta-feira (9) após o anúncio do presidente dos Estados Unidos, Donald Trump, sobre a imposição de tarifas de 50% a todos os produtos ...
Data: 2025-09-07T00:00:00+00:00
Fonte: https://noticias.r7.com/economia/mercado-da-indicios-de-incerteza-para-a-economia-brasileira-apos-carta-de-trump-10072025/
--------------------------------------------------------------------------------

NOTÍCIA 2:
Título: Ipsos-Ipec: 72% acreditam que recentes conflitos entre países podem afetar muito a economia brasileira
Resumo: Pesquisa analisou percepções da população sobre os recentes conflitos globais e sua repercussão para o Brasil. Maior apoio é aos EUA, e maior reprovação a Irã e Rússia. Pesquisa Ipsos-Ipec divulgada ...
Data: 2025-0

In [11]:
# Exemplo 3: Notícias sobre esportes
query3 = "Busque notícias recentes sobre a Copa do Mundo"
resultado3 = executar_consulta_com_monitoramento(query3)


Consultando: Busque notícias recentes sobre a Copa do Mundo




NOTÍCIAS ENCONTRADAS:
Encontradas 2 notícias sobre 'Busque notícias recentes sobre a Copa do Mundo':

NOTÍCIA 1:
Título: Cristiano Ronaldo anuncia que Copa do Mundo de 2026 será a sua última
Resumo: Craque da seleção de Portugal, Cristiano Ronaldo estreou em Copas na edição de 2006, quando os lusos chegaram à semifinal do torneio ...
Data: 2025-11-11T00:00:00+00:00
Fonte: https://www.metropoles.com/esportes/cristiano-ronaldo-anuncia-que-copa-do-mundo-de-2026-sera-a-sua-ultima
--------------------------------------------------------------------------------

NOTÍCIA 2:
Título: Copa do Mundo 2026 Canadá México Estados Unidos [futebol]
Resumo: Veja as reportagens especias sobre Copa do Mundo 2026 Canadá México Estados Unidos [futebol] no canal de tópicos do Estadão.
Data: 2025-11-16T11:16:53+00:00
Fonte: https://www.estadao.com.br/tudo-sobre/copa-do-mundo-2026-canada-mexico-estados-unidos-futebol/
--------------------------------------------------------------------------------



ANÁLISE 

**Desenvolvido como trabalho da disciplina de Inteligência Artificial**

Grupo: Gabriel H, Alan de L, Otavio A, Taiyo T, Matheus Ar.